In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_nitk_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,Govt Job: National Institute of Technology Kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0,0,NaN,0.0000,Neutral,2
1,1,Govt Job: National Institute of Technology Kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0,0,NaN,0.0000,Neutral,2
2,2,Our champ in a hero.. @ National Institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0,0,NaN,0.7717,Positive,5
3,3,தீபங்கள் பேசும்.. @ National Institute of Tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0,0,NaN,0.0000,Neutral,2
4,4,Research Associate (#ME / http://M.Tech .) In ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0,0,NaN,0.0000,Neutral,2
5,5,Our champ in a hero.. @ National Institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0,0,NaN,0.7717,Positive,5
6,6,தீபங்கள் பேசும்.. @ National Institute of Tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0,0,NaN,0.0000,Neutral,2
7,7,Research Associate (#ME / http://M.Tech .) In ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0,0,NaN,0.0000,Neutral,2
8,8,"at National Institute of Technology, Karnataka...",Apurba Mohaptra.,1153715911276335104,2019-07-23 17:18:02,[],ApurbaMohaptra,0,0,NaN,0.0000,Neutral,2
9,9,#Workshop Department of applied mechanics and ...,Jaideep Shenoy 🇮🇳,1152790966178201600,2019-07-21 04:02:38,['Workshop'],jaideepSTOI,1,1,NaN,0.3400,Positive,4


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)

In [6]:
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,Govt Job: National Institute of Technology Kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000
1,Govt Job: National Institute of Technology Kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000
2,Our champ in a hero.. @ National Institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,5,0.000000,0.000000
3,தீபங்கள் பேசும்.. @ National Institute of Tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000
4,Research Associate (#ME / http://M.Tech .) In ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000
5,Our champ in a hero.. @ National Institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,5,0.000000,0.000000
6,தீபங்கள் பேசும்.. @ National Institute of Tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000
7,Research Associate (#ME / http://M.Tech .) In ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000
8,"at National Institute of Technology, Karnataka...",Apurba Mohaptra.,1153715911276335104,2019-07-23 17:18:02,[],ApurbaMohaptra,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000
9,#Workshop Department of applied mechanics and ...,Jaideep Shenoy 🇮🇳,1152790966178201600,2019-07-21 04:02:38,['Workshop'],jaideepSTOI,1.0,1.0,NaN,0.3400,Positive,4,0.001477,0.000151


In [7]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [8]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [9]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [10]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [11]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k..."
1,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k..."
2,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,5,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,..."
3,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech..."
4,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec..."
5,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,5,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,..."
6,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech..."
7,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec..."
8,"at national institute of technology, karnataka...",Apurba Mohaptra.,1153715911276335104,2019-07-23 17:18:02,[],ApurbaMohaptra,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,at National Institute of Technology Karnataka …,"[national, institute, technology, karnataka]"
9,#workshop department of applied mechanics and ...,Jaideep Shenoy 🇮🇳,1152790966178201600,2019-07-21 04:02:38,['Workshop'],jaideepSTOI,1.0,1.0,NaN,0.3400,Positive,4,0.001477,0.000151,0.094670,#Workshop Department of applied mechanics and ...,"[workshop, department, applied, mechanics, hyd..."


In [12]:
nltk.download('averaged_perceptron_tagger')
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Manam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k...","[(govt, NNS), (job, NN), (national, JJ), (inst..."
1,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k...","[(govt, NNS), (job, NN), (national, JJ), (inst..."
2,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,5,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,...","[(champ, NN), (hero, NN), (national, JJ), (ins..."
3,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech...","[(த, JJ), (பங, NNP), (கள, NNP), (ப, NNP), (ச, ..."
4,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec...","[(research, NN), (associate, JJ), (national, J..."
5,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,5,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,...","[(champ, NN), (hero, NN), (national, JJ), (ins..."
6,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech...","[(த, JJ), (பங, NNP), (கள, NNP), (ப, NNP), (ச, ..."
7,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec...","[(research, NN), (associate, JJ), (national, J..."
8,"at national institute of technology, karnataka...",Apurba Mohaptra.,1153715911276335104,2019-07-23 17:18:02,[],ApurbaMohaptra,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,at National Institute of Technology Karnataka …,"[national, institute, technology, karnataka]","[(national, JJ), (institute, NN), (technology,..."
9,#workshop department of applied mechanics and ...,Jaideep Shenoy 🇮🇳,1152790966178201600,2019-07-21 04:02:38,['Workshop'],jaideepSTOI,1.0,1.0,NaN,0.3400,Positive,4,0.001477,0.000151,0.094670,#Workshop Department of applied mechanics and ...,"[workshop, department, applied, mechanics, hyd...","[(workshop, NN), (department, NN), (applied, V..."


In [13]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k...","[(govt, NNS), (job, NN), (national, JJ), (inst...","[govt, job, institute, technology, recruitment..."
1,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k...","[(govt, NNS), (job, NN), (national, JJ), (inst...","[govt, job, institute, technology, recruitment..."
2,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,5,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,...","[(champ, NN), (hero, NN), (national, JJ), (ins...","[champ, hero, institute, technology, karnataka]"
3,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech...","[(த, JJ), (பங, NNP), (கள, NNP), (ப, NNP), (ச, ...","[பங, கள, ப, ச, ம, institute, technology, karna..."
4,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec...","[(research, NN), (associate, JJ), (national, J...","[research, institute, technology, karnataka]"
5,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,5,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,...","[(champ, NN), (hero, NN), (national, JJ), (ins...","[champ, hero, institute, technology, karnataka]"
6,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech...","[(த, JJ), (பங, NNP), (கள, NNP), (ப, NNP), (ச, ...","[பங, கள, ப, ச, ம, institute, technology, karna..."
7,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec...","[(research, NN), (associate, JJ), (national, J...","[research, institute, technology, karnataka]"
8,"at national institute of technology, karnataka...",Apurba Mohaptra.,1153715911276335104,2019-07-23 17:18:02,[],ApurbaMohaptra,0.0,0.0,NaN,0.0000,Neutral,2,0.000000,0.000000,0.000000,at National Institute of Technology Karnataka …,"[national, institute, technology, karnataka]","[(national, JJ), (institute, NN), (technology,...","[institute, technology, karnataka]"
9,#workshop department of applied mechanics and ...,Jaideep Shenoy 🇮🇳,1152790966178201600,2019-07-21 04:02:38,['Workshop'],jaideepSTOI,1.0,1.0,NaN,0.3400,Positive,4,0.001477,0.000151,0.094670,#Workshop Department of applied mechanics and ...,"[workshop, department, applied, mechanics, hyd...","[(workshop, NN), (department, NN), (applied, V...","[workshop, departm

In [14]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,govt
1,job
2,institute
3,technology
4,recruitment
5,apply
6,research
7,posts
8,twitter
9,com


In [15]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,govt,govt,52
1,job,job,52
2,institute,institute,230
3,technology,technology,234
4,recruitment,recruitment,62
5,apply,apply,50
6,research,research,92
7,posts,posts,36
8,twitter,twitter,110
9,com,com,108


In [16]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
3,technology,technology,234
2,institute,institute,230
13,karnataka,karnataka,156
8,twitter,twitter,110
9,com,com,108
6,research,research,92
4,recruitment,recruitment,62
0,govt,govt,52
1,job,job,52
5,apply,apply,50


In [17]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [18]:
#list of irreleveant tweet topics
l=['shark','carcass','bitcoin','coingeek','lisbon']
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
120,#mangaluru: a whale shark was washed ashore ne...,DailyaddaaNews,1145031339638308864,2019-06-29 18:08:39,"['Mangaluru', 'Karnataka']",Dailyaddaa,0.0,0.0,NaN,0.4939,Positive,4,0.000000,0.000000,0.000000,#Mangaluru A whale shark was washed ashore nea...,"[mangaluru, whale, shark, washed, ashore, near...","[(mangaluru, NNS), (whale, JJ), (shark, NN), (...","[mangaluru, shark, institute, technology, karn...",2
121,"watch - #cambriansv, lisbon - coingeek origi...",CoinGeek,1254710646098341888,2020-04-27 09:55:04,"['CambrianSV', 'Bitcoin', 'Bootcamp', 'develop...",RealCoinGeek,247.0,2571.0,NaN,0.0000,Neutral,2,0.364845,0.388427,37.427784,WATCH #CambrianSV Lisbon CoinGeek Originals Fi...,"[watch, cambriansv, lisbon, coingeek, original...","[(watch, NN), (cambriansv, NN), (lisbon, NN), ...","[watch, cambriansv, lisbon, coingeek, original...",3
122,mangaluru: a whale shark was washed ashore nea...,tv9gujarati,1145030390454738944,2019-06-29 18:04:53,"['Karnataka', 'TV9Newspic']",tv9gujarati,0.0,5.0,NaN,0.0000,Neutral,2,0.000000,0.000755,0.030216,Mangaluru A whale shark was washed ashore near...,"[mangaluru, whale, shark, washed, ashore, near...","[(mangaluru, NNS), (whale, JJ), (shark, NN), (...","[mangaluru, shark, institute, technology, karn...",2
123,mangaluru: a whale shark was washed ashore nea...,ANI,1145029340540952576,2019-06-29 18:00:43,['Karnataka'],ANI,134.0,550.0,NaN,0.0000,Neutral,2,0.197932,0.083094,15.199688,Mangaluru A whale shark was washed ashore near...,"[mangaluru, whale, shark, washed, ashore, near...","[(mangaluru, NNS), (whale, JJ), (shark, NN), (...","[mangaluru, shark, institute, technology, karn...",2
129,#mangaluru: a whale shark was washed ashore ne...,DailyaddaaNews,1145031339638308864,2019-06-29 18:08:39,"['Mangaluru', 'Karnataka']",Dailyaddaa,0.0,0.0,NaN,0.4939,Positive,4,0.000000,0.000000,0.000000,#Mangaluru A whale shark was washed ashore nea...,"[mangaluru, whale, shark, washed, ashore, near...","[(mangaluru, NNS), (whale, JJ), (shark, NN), (...","[mangaluru, shark, institute, technology, karn...",2
131,mangaluru: a whale shark was washed ashore nea...,tv9gujarati,1145030390454738944,2019-06-29 18:04:53,"['Karnataka', 'TV9Newspic']",tv9gujarati,0.0,5.0,NaN,0.0000,Neutral,2,0.000000,0.000755,0.030216,Mangaluru A whale shark was washed ashore near...,"[mangaluru, whale, shark, washed, ashore, near...","[(mangaluru, NNS), (whale, JJ), (shark, NN), (...","[mangaluru, shark, institute, technology, karn...",2
132,mangaluru: a whale shark was washed ashore nea...,ANI,1145029340540952576,2019-06-29 18:00:43,['Karnataka'],ANI,134.0,550.0,NaN,0.0000,Neutral,2,0.197932,0.083094,15.199688,Mangaluru A whale shark was washed ashore near...,"[mangaluru, whale, shark, washed, ashore, near...","[(mangaluru, NNS), (whale, JJ), (shark, NN), (...","[mangaluru, shark, institute, technology, karn...",2
185,"watch - #cambriansv, lisbon - coingeek origi...",CoinGeek,1254710646098341888,2020-04-27 09:55:04,"['CambrianSV', 'Bitcoin', 'Bootcamp', 'develop...",RealCoinGeek,247.0,2571.0,NaN,0.0000,Neutral,2,0.364845,0.388427,37.427784,WATCH #CambrianSV Lisbon CoinGeek Originals Fi...,"[watch, cambriansv, lisbon, coingeek, original...","[(watch, NN), (cambriansv, NN), (lisbon, NN), ...","[watch, cambriansv, lisbon, coingeek, original...",3
193,"watch - #cambriansv, lisbon - coingeek origi...",CoinGeek,1254710646098341888,2020-04-27 09:55:04,"['CambrianSV', 'Bitcoin', 'Bootcamp', 'develop...",RealCoinGeek,247.0,2571.0,NaN,0.0000,Neutral,2,0.364845,0.388427,37.427784,WATCH #CambrianSV Lisbon CoinGeek Originals Fi...,"[watch, cambriansv, lisbon, coingeek, original...","[(watch, NN), (cambriansv, NN), (lisbon, NN), ...","[watch, cambriansv, lisbon, coingeek, original...",3
204,"watch - #cambriansv, lisbon - coingeek origi...",CoinGeek,12547106460

In [19]:
def convert(score):
    OldRange = 2  
    NewRange = 4  
    NewValue = (((score + 1) * NewRange) / OldRange) + 1
    return NewValue
df['score'] = df['compound'].apply(lambda x: convert(x))
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
0,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,3.0000,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k...","[(govt, NNS), (job, NN), (national, JJ), (inst...","[govt, job, institute, technology, recruitment...",0
1,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,Neutral,3.0000,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k...","[(govt, NNS), (job, NN), (national, JJ), (inst...","[govt, job, institute, technology, recruitment...",0
2,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,4.5434,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,...","[(champ, NN), (hero, NN), (national, JJ), (ins...","[champ, hero, institute, technology, karnataka]",0
3,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,3.0000,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech...","[(த, JJ), (பங, NNP), (கள, NNP), (ப, NNP), (ச, ...","[பங, கள, ப, ச, ம, institute, technology, karna...",0
4,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,3.0000,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec...","[(research, NN), (associate, JJ), (national, J...","[research, institute, technology, karnataka]",0
5,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,Positive,4.5434,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,...","[(champ, NN), (hero, NN), (national, JJ), (ins...","[champ, hero, institute, technology, karnataka]",0
6,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,Neutral,3.0000,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech...","[(த, JJ), (பங, NNP), (கள, NNP), (ப, NNP), (ச, ...","[பங, கள, ப, ச, ம, institute, technology, karna...",0
7,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,Neutral,3.0000,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec...","[(research, NN), (associate, JJ), (national, J...","[research, institute, technology, karnataka]",0
8,"at national institute of technology, karnataka...",Apurba Mohaptra.,1153715911276335104,2019-07-23 17:18:02,[],ApurbaMohaptra,0.0,0.0,NaN,0.0000,Neutral,3.0000,0.000000,0.000000,0.000000,at National Institute of Technology Karnataka …,"[national, institute, technology, karnataka]","[(national, JJ), (institute, NN), (technology,...","[institute, technology, karnataka]",0
9,#workshop department of applied mechanics and ...,Jaideep Shenoy 🇮🇳,1152790966178201600,2019-07-21 04:02:38,['Workshop'],jaideepSTOI,1.0,1.0,NaN,0.3400,Positive,3.6800,0.001477,0.000151,0.094670,#Workshop Department of applied mechanics and ...,"[workshop, department, applied, mechanics,

In [20]:
def assignpred(n):
    if n>1 and n<2.9:
        return max((n-1),1)
    if n>3.1 and n<5:
        return min((n+1),5)
    else:
        return n

In [21]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [22]:
df['new_score']=df['score']

In [23]:
df_eng1=df[df.engagement==1]

In [24]:
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
df[df.engagement==1]=df_eng1

In [26]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [27]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [28]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k...","[(govt, NNS), (job, NN), (national, JJ), (inst...","[govt, job, institute, technology, recruitment...",0,0,3.0000
1,govt job: national institute of technology kar...,Engineer Feed,1190895567943024640,2019-11-03 07:36:43,[],theengineerfeed,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,Govt Job National Institute of Technology Karn...,"[govt, job, national, institute, technology, k...","[(govt, NNS), (job, NN), (national, JJ), (inst...","[govt, job, institute, technology, recruitment...",0,0,3.0000
2,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,...,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,...","[(champ, NN), (hero, NN), (national, JJ), (ins...","[champ, hero, institute, technology, karnataka]",0,0,4.5434
3,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech...","[(த, JJ), (பங, NNP), (கள, NNP), (ப, NNP), (ச, ...","[பங, கள, ப, ச, ம, institute, technology, karna...",0,0,3.0000
4,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec...","[(research, NN), (associate, JJ), (national, J...","[research, institute, technology, karnataka]",0,0,3.0000
5,our champ in a hero.. @ national institute of ...,Vinoth Srinivasan,1205878423895318528,2019-12-14 15:53:15,[],withluv_vino,0.0,0.0,NaN,0.7717,...,0.000000,0.000000,0.000000,Our champ in a hero. National Institute of Tec...,"[champ, hero, national, institute, technology,...","[(champ, NN), (hero, NN), (national, JJ), (ins...","[champ, hero, institute, technology, karnataka]",0,0,4.5434
6,தீபங்கள் பேசும்.. @ national institute of tech...,Vinoth Srinivasan,1204749034008637440,2019-12-11 13:05:27,[],withluv_vino,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,தீபங்கள பேசும். National Institute of Technolo...,"[த, பங, கள, ப, ச, ம, national, institute, tech...","[(த, JJ), (பங, NNP), (கள, NNP), (ப, NNP), (ச, ...","[பங, கள, ப, ச, ம, institute, technology, karna...",0,0,3.0000
7,research associate (#me / http://m.tech .) in ...,Sarkari-Job,1204301833189109760,2019-12-10 07:28:26,['ME'],sarkarijobind,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,Research Associate (#ME . In National Institut...,"[research, associate, national, institute, tec...","[(research, NN), (associate, JJ), (national, J...","[research, institute, technology, karnataka]",0,0,3.0000
8,"at national institute of technology, karnataka...",Apurba Mohaptra.,1153715911276335104,2019-07-23 17:18:02,[],ApurbaMohaptra,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,at National Institute of Technology Karnataka …,"[national, institute, technology, karnataka]","[(national, JJ), (institute, NN), (technology,...","[institute, technology, karnataka]",0,0,3.0000
9,#workshop department of applied mechanics and ...,Jaideep Shenoy 🇮🇳,1152790966178201600,2019-07-21 04:02:38,['Workshop'],jaideepSTOI,1.0,1.0,NaN,0.3400,...,0.001477,0.000151,0.094670,#Workshop Department of applied mechanics and ...,"[workshop, department, applied, mechanics, hyd...","[(works

In [29]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/NITKWeightedSentiment.csv')